<a href="https://colab.research.google.com/github/tanjatang/Data-Analyst/blob/main/%E8%B7%A8%E5%A2%83%E7%94%B5%E5%95%86%E9%80%89%E5%93%81%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 第一步，导入必要的包

我们用到的主要分为三大类：
数据导入及处理、特征工程及挖掘、可视化

In [1]:
#载入必要的包
import pandas as pd
import numpy as np
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
# 忽略不必要的warnings
warnings.filterwarnings('ignore')

# 作图
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=2.5)


## 第二步，数据录入


In [2]:
#数据
df=pd.read_csv('lesson0.csv',encoding='gbk')
df.head()

,class0,price,rank1_list,stars,reviews,delivery,rank_big,brand,stock
0,x1,180.00,1.0,4.4,6442.0,Sold by Furbo and Fulfilled by Amazon.,287.0,Furbo,In Stock.
1,x1,32.99,2.0,3.9,1839.0,Sold by Trekking tribe and Fulfilled by Amazon.,1310.0,TOOGE,In Stock.
2,x1,29.99,3.0,4.2,2032.0,Sold by FORamerica and Fulfilled by Amazon.,0.0,BOOCOSA-US,In Stock.
3,x1,139.95,4.0,4.4,376.0,Sold by Petkey and Fulfilled by Amazon.,8004.0,WOPET,In Stock.
4,x1,29.99,5.0,3.8,322.0,Sold by VI-SON and Fulfilled by Amazon.,9203.0,VINSION,In Stock.


## 第三步 数据处理

In [3]:
# 数值化数据的描述
df.describe()

,price,rank1_list,stars,reviews,rank_big
count,50.000000,50.00000,50.000000,50.00000,50.000000
mean,92.508600,25.50000,3.996000,316.08000,36342.140000
std,70.207806,14.57738,0.789461,971.56129,36688.471828
min,0.000000,1.00000,0.000000,0.00000,0.000000
25%,32.315000,13.25000,3.800000,14.00000,0.000000
50%,69.010000,25.50000,4.100000,57.00000,25013.500000
75%,141.480000,37.75000,4.400000,146.00000,70853.750000
max,280.490000,50.00000,5.000000,6442.00000,107079.000000


In [4]:
# 空缺值判断，其中，价格、排名是我们重点关注的
df.loc[df['price']==0,'price']=None
df.loc[df['rank_big']==0,'rank_big']=None
df.isnull().any()

class0        True
price         True
rank1_list    True
stars         True
reviews       True
delivery      True
rank_big      True
brand         True
stock         True
dtype: bool

### 空缺值处理

针对排名有缺失的，我们用*临近值*来进行填充，在pandas中，会用到`DataFrame.fillna(method='backfill')`函数，，其中`method`可以是`backfill/bfill`-空缺值后面的值替换缺失值；`pad/ffill`-空缺值前面的值替换缺失值；直接赋值`fillna(0)`，用0来填补 




In [10]:
# 缺货的删掉
df=df[df['stock']!='Currently unavailable.']
# 删除空缺的
df=df.dropna(axis=0, how='any', subset=['brand','price','stock'], inplace=False)

#其中，大类排名用前后值来
df['rank_big_up']=df['rank_big']
df['rank_big_down']=df['rank_big']
df['rank_big']=(df['rank_big_up'].fillna(method='pad')+df['rank_big_down'].fillna(method='backfill'))/2

#对delivery的填补
df['delivery']=df['delivery'].fillna('none')

In [8]:
df.isnull().any()

class0           False
price            False
rank1_list       False
stars            False
reviews          False
delivery         False
rank_big         False
brand            False
stock            False
rank_big_up      False
rank_big_down    False
dtype: bool

### 连续数值变量离散化
包括：
- 小类排名：小于10，大于10
- reviews评论：按照50,100,300,1000,1500,2000和大于2000
- 价格：按照0-180内每隔20划分，大于180是一个
- stars：0,1,2,3,4,5

主要用到了pandas中的`pd.cut(Series,bins,labels)`函数，其中，`bins`是区间，`labels`是区间的标签名称

在这里，我们用到了pandas处理数据的最常用工具之一`apply`函数，它是pandas里自由度最高的函数之一，也是我们在用pandas处理数据时使用频率最高的函数，它的作用是高效的将某一个方法（函数）作用于pandas某一行或是某一列的每一个值。

用法如下，`DataFrame.apply(func, axis=0, broadcast=False, raw=False, reduce=None, args=(), **kwds)`，其中，`func`是函数，可以是pandas自带的，也可以是自己编写的，也可以是lambda匿名函数，这三种方法，我们在之后的数据处理过程中都会用到。axis是作用于行与列，`axis=1`，则是把一行数据传入函数中，`axis=0`是把一列数据传入函数中。

In [11]:
df_clean=df.copy()
# 
df_clean['rank1_list_cut']=df_clean['rank1_list'].apply(lambda x:'<=10'if x<=10 else '>10')
#连续数值变量离散化
#分项看 reveiws
df_clean['reviews_cut']=pd.cut(df_clean['reviews'].apply(int),bins=[0,50,100,300,1000,1500,2000,1000000],right=False,labels=['0-50','50-100','100-300','300-1000','1000-1500','1500-2000','>2000'])
# price
df_clean['price_cut']=pd.cut(df_clean['price'].apply(int),bins=[0,20,40,60,80,100,120,140,160,180,1000000],right=False,labels=['0-20','20-40','40-60','60-80','80-100','100-120','120-140','140-160','160-180','>180'])
# stars
df_clean['stars_cut']=pd.cut(df_clean['stars'].apply(float),bins=[0,1,2,3,4,5,6],right=False,labels=['0-1','1-2','2-3','3-4','4-5','5'])
#排名
df_clean['rank_big_cut']=pd.cut(df_clean['rank_big'].apply(float),bins=[0,500,1000,5000,10000,50000,100000,10000000],right=False,labels=['0-500','500-1000','1000-5000','5000-10000','10000-50000','50000-100000','>100000'])

In [12]:
df_clean.head()

,class0,price,rank1_list,stars,reviews,delivery,rank_big,brand,stock,rank_big_up,rank_big_down,rank1_list_cut,reviews_cut,price_cut,stars_cut,rank_big_cut
0,x1,180.00,1.0,4.4,6442.0,Sold by Furbo and Fulfilled by Amazon.,287.0,Furbo,In Stock.,287.0,287.0,<=10,>2000,>180,4-5,0-500
1,x1,32.99,2.0,3.9,1839.0,Sold by Trekking tribe and Fulfilled by Amazon.,1310.0,TOOGE,In Stock.,1310.0,1310.0,<=10,1500-2000,20-40,3-4,1000-5000
2,x1,29.99,3.0,4.2,2032.0,Sold by FORamerica and Fulfilled by Amazon.,4657.0,BOOCOSA-US,In Stock.,4657.0,4657.0,<=10,>2000,20-40,4-5,1000-5000
3,x1,139.95,4.0,4.4,376.0,Sold by Petkey and Fulfilled by Amazon.,8004.0,WOPET,In Stock.,8004.0,8004.0,<=10,300-1000,120-140,4-5,5000-10000
4,x1,29.99,5.0,3.8,322.0,Sold by VI-SON and Fulfilled by Amazon.,9203.0,VINSION,In Stock.,9203.0,9203.0,<=10,300-1000,20-40,3-4,5000-10000


由于此处的离散变量带有序列性质，后期我们会根据它们的值来进行计算，所以，我们需要用`df.cat.codes`来获取序列变量的值

其中，根据之前讲解，reviews和排名（rank）是越高，对运营越不利；price和stars则相反，所以我们需要把它们同向化

In [14]:
df_clean['price_cut_n']=df_clean['price_cut'].cat.codes+1
df_clean['reviews_cut_n']=7-df_clean['reviews_cut'].cat.codes
df_clean['price_cut_n']=df_clean['price_cut'].cat.codes+1
df_clean['stars_cut_n']=df_clean['stars_cut'].cat.codes+1
df_clean['rank_big_cut_n']=7-df_clean['rank_big_cut'].cat.codes
df_clean.head()

,class0,price,rank1_list,stars,reviews,delivery,rank_big,brand,stock,rank_big_up,rank_big_down,rank1_list_cut,reviews_cut,price_cut,stars_cut,rank_big_cut,price_cut_n,reviews_cut_n,stars_cut_n,rank_big_cut_n
0,x1,180.00,1.0,4.4,6442.0,Sold by Furbo and Fulfilled by Amazon.,287.0,Furbo,In Stock.,287.0,287.0,<=10,>2000,>180,4-5,0-500,10,1,5,7
1,x1,32.99,2.0,3.9,1839.0,Sold by Trekking tribe and Fulfilled by Amazon.,1310.0,TOOGE,In Stock.,1310.0,1310.0,<=10,1500-2000,20-40,3-4,1000-5000,2,2,4,5
2,x1,29.99,3.0,4.2,2032.0,Sold by FORamerica and Fulfilled by Amazon.,4657.0,BOOCOSA-US,In Stock.,4657.0,4657.0,<=10,>2000,20-40,4-5,1000-5000,2,1,5,5
3,x1,139.95,4.0,4.4,376.0,Sold by Petkey and Fulfilled by Amazon.,8004.0,WOPET,In Stock.,8004.0,8004.0,<=10,300-1000,120-140,4-5,5000-10000,7,4,5,4
4,x1,29.99,5.0,3.8,322.0,Sold by VI-SON and Fulfilled by Amazon.,9203.0,VINSION,In Stock.,9203.0,9203.0,<=10,300-1000,20-40,3-4,5000-10000,2,4,4,4


## 第四步 确定并实现指标体系
### 指标体系的计算

在这里，我们需要引入一个新的函数groupby，以及apply的加强版agg函数，通常，apply是与groupby连用的

groupby+apply，可以极大的提高我们处理数据的效率，刚开始可能大家用的不熟练，但是等熟悉之后，就会发现它的好处

另外，就是apply的进阶版，agg函数，它是能指定轴上的一个或多个操作进行聚合，同样默认为是列轴。常见用法：`df_columns=df.agg({'col1':[func1,func2],'col2':[func3,func4]})`

In [18]:
df_clean['brand'].value_counts()

Petcube         4
SKYMEE          4
DOGNESS         4
TOOGE           3
VINSION         2
PetChatz        2
DOGCOOL         2
VIMTAG          2
AONESY          2
hosecurity      2
DEYAN           2
TAOCOCO         1
RVA Camworks    1
VSTARCAM        1
GevTa           1
Homenhancing    1
BOOCOSA-US      1
DADYPET         1
Furbo           1
NPET            1
TESECU          1
Tiscen          1
YI              1
Mliyam          1
WOPET           1
FitBark         1
RV PetSafety    1
Iseebiz         1
DEATTI          1
RCA             1
Pawbo           1
Name: brand, dtype: int64

- 构造函数，并计算各个指标

![lesson1-3-2.png](attachment:lesson1-3-2.png)

- 求集中度，逻辑是频数最高的类别

用到了`Series.value_counts()`函数，其中`ascending=False`是倒序排列，取最大值即类别最高的

- 求分位数？为什么用分位数而不直接用平均值？

用到了`quantile([0.2,0.4,0.8])`函数，其中列表里是在不同百分比下所有数值由小到大排列后前百分位数的数字


In [19]:
# 拉平之前
def get_most_count2(n):
    counts=n.value_counts(ascending=False)
    name=n.value_counts(ascending=False).index[0]
    num=n.value_counts(ascending=False).values[0]
    r=num/sum(counts)
    return name,num,r
    
def get_quantile(n):
    return list(n.quantile([0,0.2,0.5,0.8,1]))


feature={
            'class0':'count',
            'price_cut_n':get_most_count2,
            'price':['mean','median','max','min',get_quantile],
            'reviews_cut_n':get_most_count2,
            'reviews':['mean','median','max','min',get_quantile],
            'stars_cut_n':get_most_count2,
            'stars':['mean','median','max','min',get_quantile],
            'rank_big_cut_n':get_most_count2,
            'rank_big':['mean','median','max','min',get_quantile]
        }

df_clean[['price','stars','reviews']]=df_clean[['price','stars','reviews']].apply(pd.to_numeric)
df_sample=df_clean.groupby('class0').agg(feature)
df_sample.head()

class0  ...                                      rank_big
        count  ...                                  get_quantile
class0         ...                                              
x1         49  ...  [287.0, 18666.4, 43569.0, 82578.1, 107079.0]

[1 rows x 25 columns]

In [20]:
# 名称拉平
df_new=df_sample.copy()
for col_num,col_name in enumerate(df_sample):
    if col_name in [('price_cut_n', 'get_most_count2'),('reviews_cut_n', 'get_most_count2'),('stars_cut_n', 'get_most_count2'),('rank_big_cut_n', 'get_most_count2')]:
        p=df_sample[col_name].apply(pd.Series)
        p.columns=[col_name[0]+'_most',col_name[0]+'_most_count',col_name[0]+'_most_ratio']
        # 重新聚合
        df_new=pd.concat([df_new.drop(col_name,axis=1),p],axis=1)
    elif col_name in [('rank_big', 'get_quantile'),('price', 'get_quantile'),('reviews', 'get_quantile'),('stars', 'get_quantile')]:
        p2=df_sample[col_name].apply(pd.Series)
        p2.columns=[col_name[0]+'_0',col_name[0]+'_20',col_name[0]+'_40',col_name[0]+'_80',col_name[0]+'_100']
        df_new=pd.concat([df_new.drop(col_name,axis=1),p2],axis=1)
name=[]
for i in df_new.columns:
    if isinstance(i,tuple):
        name.append(i[0]+'-'+i[1])
    else:
        name.append(i)
df_new.columns=[i+'_top_50' for i in name]
df_new

,class0-count_top_50,price-mean_top_50,price-median_top_50,price-max_top_50,price-min_top_50,reviews-mean_top_50,reviews-median_top_50,reviews-max_top_50,reviews-min_top_50,stars-mean_top_50,stars-median_top_50,stars-max_top_50,stars-min_top_50,rank_big-mean_top_50,rank_big-median_top_50,rank_big-max_top_50,rank_big-min_top_50,price_cut_n_most_top_50,price_cut_n_most_count_top_50,price_cut_n_most_ratio_top_50,price_0_top_50,price_20_top_50,price_40_top_50,price_80_top_50,price_100_top_50,reviews_cut_n_most_top_50,reviews_cut_n_most_count_top_50,reviews_cut_n_most_ratio_top_50,reviews_0_top_50,reviews_20_top_50,reviews_40_top_50,reviews_80_top_50,reviews_100_top_50,stars_cut_n_most_top_50,stars_cut_n_most_count_top_50,stars_cut_n_most_ratio_top_50,stars_0_top_50,stars_20_top_50,stars_40_top_50,stars_80_top_50,stars_100_top_50,rank_big_cut_n_most_top_50,rank_big_cut_n_most_count_top_50,rank_big_cut_n_most_ratio_top_50,rank_big_0_top_50,rank_big_20_top_50,rank_big_40_top_50,rank_big_80_top_50,rank_big_100_top_50
class0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
x1,49,94.396531,69.87,280.49,18.56,310.632653,57.0,6442.0,0.0,3.993878,4.1,5.0,0.0,50056.132653,43569.0,107079.0,287.0,2.0,17.0,0.346939,18.56,31.596,69.87,170.75,280.49,7.0,23.0,0.469388,0.0,8.0,57.0,180.2,6442.0,5.0,27.0,0.55102,0.0,3.7,4.1,4.4,5.0,3.0,22.0,0.44898,287.0,18666.4,43569.0,82578.1,107079.0


In [21]:
df_new['amz']=len(df_clean[df_clean.delivery=='Ships from and sold by Amazon.com.'])/len(df_clean)
df_new['delivery']=len(df_clean[df_clean['delivery'].str.contains('Fulfilled by Amazon')])/len(df_clean)

#某类别下有多少品牌
b1=df_clean['brand'].value_counts()
b2=df_clean['brand'].value_counts(normalize=True)   
df_new['brand_count']=len(b1)
df_new['brand_top_ratio']=b2[0]

df_new

,class0-count_top_50,price-mean_top_50,price-median_top_50,price-max_top_50,price-min_top_50,reviews-mean_top_50,reviews-median_top_50,reviews-max_top_50,reviews-min_top_50,stars-mean_top_50,stars-median_top_50,stars-max_top_50,stars-min_top_50,rank_big-mean_top_50,rank_big-median_top_50,rank_big-max_top_50,rank_big-min_top_50,price_cut_n_most_top_50,price_cut_n_most_count_top_50,price_cut_n_most_ratio_top_50,price_0_top_50,price_20_top_50,price_40_top_50,price_80_top_50,price_100_top_50,reviews_cut_n_most_top_50,reviews_cut_n_most_count_top_50,reviews_cut_n_most_ratio_top_50,reviews_0_top_50,reviews_20_top_50,reviews_40_top_50,reviews_80_top_50,reviews_100_top_50,stars_cut_n_most_top_50,stars_cut_n_most_count_top_50,stars_cut_n_most_ratio_top_50,stars_0_top_50,stars_20_top_50,stars_40_top_50,stars_80_top_50,stars_100_top_50,rank_big_cut_n_most_top_50,rank_big_cut_n_most_count_top_50,rank_big_cut_n_most_ratio_top_50,rank_big_0_top_50,rank_big_20_top_50,rank_big_40_top_50,rank_big_80_top_50,rank_big_100_top_50,amz,delivery,brand_count,brand_top_ratio
class0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
x1,49,94.396531,69.87,280.49,18.56,310.632653,57.0,6442.0,0.0,3.993878,4.1,5.0,0.0,50056.132653,43569.0,107079.0,287.0,2.0,17.0,0.346939,18.56,31.596,69.87,170.75,280.49,7.0,23.0,0.469388,0.0,8.0,57.0,180.2,6442.0,5.0,27.0,0.55102,0.0,3.7,4.1,4.4,5.0,3.0,22.0,0.44898,287.0,18666.4,43569.0,82578.1,107079.0,0.0,0.897959,31,0.081633


In [22]:
# 主要特征筛选
#选择特征
market_demond_list=['rank_big_80_top_50']
#市场竞争 
# 选择特征
market_chan_list=['reviews-median_top_50','brand_count','brand_top_ratio']
#商品相关
product_info_list=['amz','delivery','stars-mean_top_50','price-median_top_50']
df_final=df_new[market_chan_list+market_demond_list+product_info_list]

In [23]:
df_final

,reviews-median_top_50,brand_count,brand_top_ratio,rank_big_80_top_50,amz,delivery,stars-mean_top_50,price-median_top_50
class0,,,,,,,,
x1,57.0,31,0.081633,82578.1,0.0,0.897959,3.993878,69.87


### 指标体系的进一步处理

In [24]:
data=df_final.copy()

- 指标同向化

In [25]:
#指标同向化，取反
def get_inverse(n):
    return -n

#指标同向化
data[['amz','brand_count','brand_count','reviews-median_top_50']]=data[['amz','brand_count','brand_count','reviews-median_top_50']].apply(get_inverse)

In [26]:
data

,reviews-median_top_50,brand_count,brand_top_ratio,rank_big_80_top_50,amz,delivery,stars-mean_top_50,price-median_top_50
class0,,,,,,,,
x1,-57.0,-31,0.081633,82578.1,-0.0,0.897959,3.993878,69.87


- 数据标准化

这里用到了python的另一个包`sklearn`中的`preprocessing`的预处理数据工具箱，用它我们可以实现数据的标准化，在这里我们用到了`Min-Max`标准化，利用公式 $$ y_i=\frac{x_i-min(x)}{max(x)-min(x)}$$ 

对原始数据的线性变换，使结果落到$ [0,1]$ 区间，可以保留极大极小值。

In [27]:
minmax_scaler = preprocessing.MinMaxScaler()  # 建立MinMaxScaler模型对象

### 指标权重确定

- 主观评价法

In [28]:
market_demond=0.3
market_chan=0.4
product_info=0.3
weight_dict={
        #市场需求:权重和为1
        'rank_big_cut_n_most_top_10':market_demond*0.6,
        'rank_big_80_top_50':market_demond*0.4,
        
        #市场竞争:权重和为1
        'reviews-median_top_50':market_chan*0.2,
        'reviews_cut_n_most_top_10':market_chan*0.3,
        'brand_count':market_chan*0.3,
        'brand_top_ratio':market_chan*0.2,
        
        #商品相关:权重和为1
        'amz':product_info*0.2,
        'delivery':product_info*0.15,
        'stars_cut_n_most_top_10':product_info*0.05,
        'stars-mean_top_50':product_info*0.15,
        'price-median_top_50':market_demond*0.15,
        'price_cut_n_most_top_10':market_demond*0.2,
        'price_80_top_10':market_demond*0.1
        }
#专家赋权
W1 = pd.DataFrame(weight_dict,index=['expert_w']).T
W1['expert_w'].sum()

1.0

## 第五步 综合指标法
将分数与权重相乘得到最终综合得分

In [29]:
def get_score(df,name,weight='expert_w',col='final_score'):
    '''
    df:数据框
    name:指标列表
    weight:权重选择
    col:指标列表名字
    '''
    # name是一级分类，也可以是所有的
    t1=df[name]
    #用到的权重体系
    W=W1.loc[name,weight]
    W=np.array(W)
    #计算并保存分数
    U=[]
    for i in range(1,len(t1)+1):
        # 获取样本各个指标的值
        y=t1[i-1:i].values
        # 综合得分
        u=np.sum(y*W)*100
        U.append(u)
    U=pd.DataFrame(U,index=list(t1.index))
    U.columns=[col]
    U[col+' rank']=U[col].rank(ascending=False)
    return U

In [30]:
data_final=get_score(data,market_demond_list,'expert_w','score_market_demond')
data_final=data_final.join(get_score(data,market_chan_list,'expert_w','score_market_chan'))
data_final=data_final.join(get_score(data,product_info_list,'expert_w','score_product_info'))
data_final['score']=data_final['score_market_demond']+data_final['score_market_chan']+data_final['score_product_info']
#排名，取高的
data_final['rank']=data_final['score'].rank(ascending=False)

data_final.sort_values('rank')

,score_market_demond,score_market_demond rank,score_market_chan,score_market_chan rank,score_product_info,score_product_info rank,score,rank
x1,990937.2,1.0,-827.346939,1.0,336.428265,1.0,990446.281327,1.0
